## VISIT_OCCURRENCE

See [visit_occurrence](https://ohdsi.github.io/CommonDataModel/cdm54.html#visit_occurrence).

This table records each patient's encounters with the healthcare system. It is somewhat like the fine detail of the observation_period table. The fields in the table are as follows:

```{mermaid}
erDiagram
    OMOP_VISIT_OCCURRENCE {
        integer visit_occurence_id
        integer person_id
        integer visit_concept_id
        varchar(50) visit_source_value
        integer visit_source_concept_id
        date visit_start_date
        datetime visit_start_datetime
        date visit_end_date
        datetime visit_end_datetime
        integer visit_type_concept_id
        integer provider_id
        integer care_site_id
        integer admitted_from_concept_id
        varchar(50) admitted_from_source_value
        integer discharged_to_concept_id
        varchar(50) discharged_to_source_value
        integer preceding_visit_occurrence_id
    }
```

The script that handles the transformation from the original file to OMOP format is [genomop_visit_occurrence.py](../examples/genomop_visit_occurrence.py). In this case we have to gather all those files that contain information about a patient's interaction with the health system. Not all files have this info, hence, not all files need to be taken into account.

Once we have all the interactions, it will be necessary to eliminate duplicates and those visits that contain others, e.g. punctual pharmacy interactions during a hospitalization of several days. This is done by the function `remove_overlap()` in the [general](../bps_to_omop/general.py) module.

The configuration file will be [genomop_visit_occurrence_params.yaml](../examples/genomop_visit_occurrence_params.yaml). The configuration file must have the following structure:

```YAML
input_dir: preomop/03_omop_initial/
output_dir: preomop/04_omop_intermediate/VISIT_OCCURRENCE/
input_files:
  - 02_Patologias_BPS.parquet
  - 03_MPA.parquet
transformations: # Transformation to apply to each file
  - 02_Patologias_BPS.parquet: function_name
visit_concept_dict: # For each file
  02_Patologias_BPS.parquet:
    - - single_code # Apply this value for every row
      - 9202
      - {}
  03_MPA.parquet:
    - - single_code # Apply this value for every row
      - 32036
      - {}
visit_concept_order: # Hierarchy of concepts. When one of two rows have to be removed, the top one stays
  - 9202
  - 32036
  - 0
remove_overlap:
  sorting_columns: ["person_id", "start_date", "end_date", "visit_concept_id"]
  ascending_order: [True, True, False, True]
```



The parameters are:
- `files_to_use`: Defines the files that should be used to assemble this table. The others will be ignored, even if they appear in files_list. 
- `transformations`: Defines the transformations that must be made to the file before incorporating it to the rest of the data. 
  - A clear example would be one in which the column `end_date` is not reliable or relevant. So it is replaced by a copy of start_date.
  - Transformations are performed by functions that operate on pyarrow tables. See [table_transformer.py](../external/bps_to_omop/bps_to_omop/table_transformer.py).
- `visit_concept_dict`: Dictionary indicating the order of the codes to apply and how to apply them. Right now there are three options:
    - `single_code`: Applies one code to all cases.
        - When the file has only one code.
    - `field_code`: Applies a code only to cases where the `column` column has the value `column_value`.
        - Useful when the visit type is encoded in a column inside the file.
    - `duration_code`: Applies a code only to cases where the interval between end_date and start_date is within maximum and minimum value.
        - Useful when the type of visit depends on its duration. E.g. if it is a one day visit to the hospital, it is an outpatient, if it is more than one day, it is probably an inpatient.
    - It is important to note that the order matters. At first code 0 is assigned to all rows, then the functions are applied in order. Therefore, the last entry will take precedence in case it is applied to more than one record. 
- `visit_concept_order`: Hierarchy for the codes used. This is used when grouping visits. Multiday visits cannot overlap. In case two visits occur during the same period, one must contain the other. This order identifies which visit should be kept and which should be discarded.
- `remove_overlap`: Defines the parameters to detect and proceed when two rows are overlapping. The procedure works by first sorting the rows and then removing all rows contained in a previous row. Since only the first row will be kept, order is important.
  - `sorting_columns`: Columns to be used for sorting. Usually: ["person_id", "start_date", "end_date", "visit_concept_id"]
  - `ascending_order`: Bool indicating if sorting should be ascending or not. [True, True, False, True]